# Fabric API Development

## Imports

In [1]:
import os
import sys
import uuid

from dotenv import load_dotenv

sys.path.append("../src")

load_dotenv("../.env")

True

In [2]:
from fabricengineer.api.auth import MicrosoftExtraSVC
from fabricengineer.api.fabric.client.fabric import set_global_fabric_client
from fabricengineer.api.fabric.workspace import Workspace
from fabricengineer.api.fabric.workspace.folder import WorkspaceFolder
from fabricengineer.api.fabric.workspace.items import (
    Lakehouse, Warehouse, VariableLibrary, VariableLibraryDefinition
)

In [3]:
WORKSPACE_ID = os.environ.get("WORKSPACE_ID")

MICROSOFT_TENANT_ID = os.environ.get("MICROSOFT_TENANT_ID")
SVC_MICROSOFT_FABRIC_CLIENT_ID = os.environ.get("SVC_MICROSOFT_FABRIC_CLIENT_ID")
SVC_MICROSOFT_FABRIC_SECRET_VALUE = os.environ.get("SVC_MICROSOFT_FABRIC_SECRET_VALUE")

svc = MicrosoftExtraSVC(
    tenant_id=MICROSOFT_TENANT_ID,
    client_id=SVC_MICROSOFT_FABRIC_CLIENT_ID,
    client_secret=SVC_MICROSOFT_FABRIC_SECRET_VALUE
)

assert len(svc.tenant_id) == 36
assert len(svc.client_id) == 36
assert len(svc.client_secret) == 40

In [4]:
client = set_global_fabric_client(svc)
client

## Utils

In [5]:
def item_name(prefix: str) -> str:
    name = uuid.uuid4().hex[:8].replace("-", "")
    return f"{prefix}_{name}"

## Workspaces

### Get Workspaces

In [6]:
client.workspaces.get().json()

{'value': [{'id': '312bfed8-b61c-4465-8907-ec3a32bbfd8a',
   'displayName': 'Fabric Capacity Metrics',
   'description': '',
   'type': 'Workspace'},
  {'id': '3496c8e0-ff9a-4a3a-95ac-0dc36994c20d',
   'displayName': 'FabricEngineer-py Unit-Tests [UT]',
   'description': '',
   'type': 'Workspace',
   'capacityId': 'd2223036-a08d-4750-930b-fea5c68b0d1b'}]}

In [7]:
Workspace.list()

[Workspace(item=FabricItem(fields={'displayName': 'Fabric Capacity Metrics', 'description': 'New Workspace', 'capacityId': None}, api=WorkspaceAPIData(id='312bfed8-b61c-4465-8907-ec3a32bbfd8a', displayName='Fabric Capacity Metrics', description='', type='Workspace', capacityId=None, capacityRegion=None, capacityAssignmentProgress=None, oneLakeEndpoints=WorkspaceOneLakeEndpoints(blobEndpoint=None, dfsEndpoint=None))),
 Workspace(item=FabricItem(fields={'displayName': 'FabricEngineer-py Unit-Tests [UT]', 'description': 'New Workspace', 'capacityId': None}, api=WorkspaceAPIData(id='3496c8e0-ff9a-4a3a-95ac-0dc36994c20d', displayName='FabricEngineer-py Unit-Tests [UT]', description='', type='Workspace', capacityId='d2223036-a08d-4750-930b-fea5c68b0d1b', capacityRegion=None, capacityAssignmentProgress=None, oneLakeEndpoints=WorkspaceOneLakeEndpoints(blobEndpoint=None, dfsEndpoint=None)))]

In [8]:
client.workspaces.get(WORKSPACE_ID).json()

{'id': '3496c8e0-ff9a-4a3a-95ac-0dc36994c20d',
 'displayName': 'FabricEngineer-py Unit-Tests [UT]',
 'description': '',
 'type': 'Workspace',
 'capacityId': 'd2223036-a08d-4750-930b-fea5c68b0d1b',
 'capacityRegion': 'Germany West Central',
 'oneLakeEndpoints': {'blobEndpoint': 'https://germanywestcentral-onelake.blob.fabric.microsoft.com',
  'dfsEndpoint': 'https://germanywestcentral-onelake.dfs.fabric.microsoft.com'},
 'capacityAssignmentProgress': 'Completed'}

In [9]:
Workspace.get_by_id(WORKSPACE_ID)

Workspace(item=FabricItem(fields={'displayName': 'FabricEngineer-py Unit-Tests [UT]', 'description': 'New Workspace', 'capacityId': None}, api=WorkspaceAPIData(id='3496c8e0-ff9a-4a3a-95ac-0dc36994c20d', displayName='FabricEngineer-py Unit-Tests [UT]', description='', type='Workspace', capacityId='d2223036-a08d-4750-930b-fea5c68b0d1b', capacityRegion='Germany West Central', capacityAssignmentProgress='Completed', oneLakeEndpoints=WorkspaceOneLakeEndpoints(blobEndpoint='https://germanywestcentral-onelake.blob.fabric.microsoft.com', dfsEndpoint='https://germanywestcentral-onelake.dfs.fabric.microsoft.com')))

In [10]:
Workspace.get_by_name("FabricEngineer-py Unit-Tests [UT]")

Workspace(item=FabricItem(fields={'displayName': 'FabricEngineer-py Unit-Tests [UT]', 'description': 'New Workspace', 'capacityId': None}, api=WorkspaceAPIData(id='3496c8e0-ff9a-4a3a-95ac-0dc36994c20d', displayName='FabricEngineer-py Unit-Tests [UT]', description='', type='Workspace', capacityId='d2223036-a08d-4750-930b-fea5c68b0d1b', capacityRegion=None, capacityAssignmentProgress=None, oneLakeEndpoints=WorkspaceOneLakeEndpoints(blobEndpoint=None, dfsEndpoint=None)))

### Manage Workspace

In [11]:
name = item_name("WS")
update_name = f"{name}_Updated"
ws = Workspace(
    name=name,
    description="Test Workspace"
)

ws.create()
print("CREATED:", ws.item.api)

assert Workspace.get_by_id(ws.item.api.id) is not None

ws.update(displayName=update_name)
print("UPDATED:", ws.item.api)

assert Workspace.get_by_id(ws.item.api.id).item.api.displayName == update_name

ws.delete()

try:
    ws.fetch()
    print("ERROR: found!")
except Exception as e:
    print("Successful deleted")


CREATED: WorkspaceAPIData(id='ceb07b22-13d8-42d0-9d65-16e7a1367e15', displayName='WS_eeef807c', description='Test Workspace', type='Workspace', capacityId=None, capacityRegion=None, capacityAssignmentProgress='Completed', oneLakeEndpoints=WorkspaceOneLakeEndpoints(blobEndpoint=None, dfsEndpoint=None))
UPDATED: WorkspaceAPIData(id='ceb07b22-13d8-42d0-9d65-16e7a1367e15', displayName='WS_eeef807c_Updated', description='Test Workspace', type='Workspace', capacityId=None, capacityRegion=None, capacityAssignmentProgress='Completed', oneLakeEndpoints=WorkspaceOneLakeEndpoints(blobEndpoint=None, dfsEndpoint=None))
Successful deleted
